In [ ]:
# default_exp fully_connected

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import *

from pathlib import Path
from IPython.core.debugger import set_trace
from fastai import datasets
import pickle, gzip, math, torch, matplotlib as mpl
import matplotlib.pyplot as plt
from torch import tensor

In [ ]:
#export
from fastai_id.matmul import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext=".gz")
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train, y_train, x_valid, y_valid))

def normalize(x, m, s): return (x-m)/s

In [ ]:
x_train, y_train, x_valid, y_valid = get_data()

In [ ]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(0.1304), tensor(0.3073))

In [ ]:
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [ ]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(-3.0466e-06), tensor(1.))

In [ ]:
test_close(x_train.mean(),0)
test_close(x_train.std(),1)

In [ ]:
n,m = x_train.shape
c = y_train.max() + 1
n,m,c

(50000, 784, tensor(10))

# Foundations Version

## Basic Version

In [ ]:
# num hidden
nh = 50

In [ ]:
# Work through this 
# simplified kaiming init / he init
w1 = torch.randn(m, nh) / math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh, 1) / math.sqrt(nh)
b2 = torch.zeros(1)

In [ ]:
test_close(w1.mean(),0, eps=1e-03)
test_close(w1.std(),1, eps=1)

In [ ]:
# This should be ±(0,1) (mean, std)
x_valid.mean(), x_valid.std()

(tensor(-0.0059), tensor(0.9924))

In [ ]:
def lin(x, w, b): return x@w + b

In [ ]:
def relu(x): return x.clamp_min(0.)

In [ ]:
t = relu(lin(x_train, w1, b1))

In [ ]:
# this should have mean 0 and std §
t.mean(), t.std()

(tensor(0.3785), tensor(0.5642))

In [ ]:
# kaiming init / he init for relu
w1 = torch.randn(m, nh) * math.sqrt(2/m)

In [ ]:
t = relu(lin(x_train, w1, b1))
t.mean(), t.std()

(tensor(0.5150), tensor(0.7989))

In [ ]:
#export
from torch.nn import init

In [ ]:
w1 = torch.zeros(m, nh)
init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(x_valid,w1,b1))

In [ ]:
t.mean(), t.std()

(tensor(0.6321), tensor(0.9001))

In [ ]:
# what if...
def relu(x): return x.clamp_min(0.) - 0.5

In [ ]:
# kaiming init / he init for relu
w1 = torch.randn(m, nh) * math.sqrt(2/m)
t1 = relu(lin(x_valid,w1,b1))
t1.mean(), t1.std()

(tensor(-0.0157), tensor(0.7541))

In [ ]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [ ]:
%timeit -n 10 _=model(x_valid)

4.84 ms ± 599 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
assert model(x_valid).shape == torch.Size([x_valid.shape[0],1])

## Loss function - MSE

In [ ]:
model(x_valid).shape

torch.Size([10000, 1])

In [ ]:
def mse(output, targs): return (output.squeeze(-1)-targs).pow(2).mean()

In [ ]:
y_train, y_valid = y_train.float(), y_valid.float()

In [ ]:
preds = model(x_train)

In [ ]:
preds.shape

torch.Size([50000, 1])

In [ ]:
mse(preds, y_train)

tensor(27.4734)

## Gradients and backwards pass

In [ ]:
def mse_grad(inp, targ):
    # grad of loss with respect to output of previous layer
    inp.g =  2.0 * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [ ]:
def relu_grad(inp, out):
    # grad of relu with respect to input  activations
    inp.g = (inp>0).float() * out.g

In [ ]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect ot input
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [ ]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # we don't actually need the loss in backward!
    loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [ ]:
forward_and_backward(x_train, y_train)

In [ ]:
# save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig = x_train.g.clone()

We cheat a bit and use PyTorch autograd to check our results

In [ ]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [ ]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)

In [ ]:
loss = forward(xt2, y_train)

In [ ]:
loss.backward()

In [ ]:
test_close(w22.grad, w2g, eps=1e-3)
test_close(b22.grad, b2g, eps=1e-3)
test_close(w12.grad, w1g, eps=1e-3)
test_close(b12.grad, b1g, eps=1e-3)
test_close(xt2.grad, ig, eps=1e-3)

In [ ]:
loss

tensor(27.4734, grad_fn=<MeanBackward0>)

## Refactor the model

## Layers as classes

In [ ]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.) - 0.5
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0.).float() * self.out.g

In [ ]:
class Lin():
    def __init__(self, w, b): self.w, self.b = w, b
    
    def __call__(self, inp):
        self.inp = inp
        self.out = inp @ self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [ ]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [ ]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [ ]:
w1.g, b1.g, w2.g, b2.g = [None] * 4
model = Model(w1, b1, w2, b2)

In [ ]:
%time loss = model(x_train, y_train)

CPU times: user 153 ms, sys: 61.7 ms, total: 215 ms
Wall time: 27.5 ms


In [ ]:
%time model.backward()

CPU times: user 6.21 s, sys: 7.08 s, total: 13.3 s
Wall time: 2.03 s


In [ ]:
test_close(w2g, w2.g, eps=1e-3)
test_close(b2g, b2.g, eps=1e-3)
test_close(w1g, w1.g, eps=1e-3)
test_close(b1g, b1.g, eps=1e-3)
test_close(ig, x_train.g, eps=1e-3)

In [ ]:
#hide
!nbdev_build_lib
!nbdev_build_docs

Converted 01_matmul.ipynb.
Converted 02_fully_connected.ipynb.
Converted index.ipynb.
converting: /notebooks/fastai_id/02_fully_connected.ipynb
converting /notebooks/fastai_id/index.ipynb to README.md
